## Setup


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from datagen import DataGenerator # Custom class import

Set global variables.

Paths should be formatted _exactly_ as shown.

Do not edit `TRAIN_FILES`, `VAL_FILES`, or their respective generators.


In [ ]:
SAVE_PATH = "/path/to/save/"

EPOCHS = 200
PARAMS = {"batch_size": 32,
          "views": 3,
          "planes": 500,
          "cells": 500,
          "n_channels": 3}
LEARN_RATE = 0.00001

EARLY_STOP = callbacks.EarlyStopping(min_delta=0.001, patience=5, monitor="val_loss",
                                     verbose=1, restore_best_weights=True)
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARN_RATE)

VAL_SPLIT = 0.2
FILES = [file[:-3] for file in glob("/path/to/batch/images.gz")]
np.random.shuffle(FILES)
TRAIN_FILES = FILES[int(VAL_SPLIT*len(FILES)):]
VAL_FILES = FILES[:int(VAL_SPLIT*len(FILES))]
TRAIN_GENERATOR = DataGenerator(TRAIN_FILES, **PARAMS)
VAL_GENERATOR = DataGenerator(VAL_FILES, **PARAMS)

Accelerate performance.


In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

## Create Model


Construct the model.

- Input: pixel map of a neutrino collision
  - Map the three views to the three image channels (RGB)
- Two sequences of:
  - 2x2, 32-filter Convolutional layer
  - 2x2 MaxPooling layer
- Flatten layer
- 16-neuron Dense layer
- Output: predicted energy in GeV


In [ ]:
def build_model(planes, cells, channels) -> models.Sequential:
    'Creates a model suitable for NuEnergy regression.'
    model = models.Sequential([
        layers.InputLayer(input_shape=(planes, cells, channels)),
        layers.Conv2D(32, (2,2), activation="relu"),
        layers.MaxPooling2D((2,2)),   
        layers.Conv2D(32, (2,2), activation="relu"),
        layers.MaxPooling2D((2,2)),   
        layers.Flatten(),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="softplus")
    ])
    return model

model = build_model(PARAMS["planes"], PARAMS["cells"], PARAMS["n_channels"])
model.summary()

Compile and train the model. Here, we use mean-squared error for loss.


In [ ]:
model.compile(optimizer=OPTIMIZER, loss="mean_squared_error")
history = model.fit(TRAIN_GENERATOR, validation_data=VAL_GENERATOR,
                    epochs=EPOCHS, callbacks=EARLY_STOP)

model.save(SAVE_PATH)

Plot the training & validation losses over the epochs.


In [ ]:
plt.plot(history.history["loss"], label="Train. Loss")
plt.plot(history.history["val_loss"], label="Val. Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.legend(loc="upper right")
plt.savefig(SAVE_PATH + "losses.png")